# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD004 - Metodologia de Aprendizado de Máquina Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---


<br>

## **Aula 02** - **Tópico: Comparação de Modelos Preditivos**

<br>

**Objetivo deste notebook**: Exemplificar o uso de intervalos de confiança e testes estatísticos para comparação de modelos preditivos.
<br>

---





##**Predição de risco de diabetes**

Os dados que utilizaremos neste notebook são os mesmos da Aula 01 e foram coletados no artigo de [Islam et al (2019)](https://link.springer.com/chapter/10.1007/978-981-13-8798-2_12) com o propósito de desenvolver um modelo para auxiliar no diagnóstico precoce de diabetes. O diagnóstico precoce só é possível através da avaliação adequada dos sintomas comuns e menos comuns, que podem ser observados em diferentes fases desde o início da doença até o diagnóstico. Os autores geraram um conjunto de dados com 520 instâncias, que foi coletado usando questionários diretos dos pacientes do Sylhet Diabetes Hospital em Sylhet, Bangladesh. Para realização desta atividade, não consideraremos o atributo idade (já descartado no conjunto de dados a ser baixado). Todos os atributos são binários, com respostas Sim/Não (Yes/No).



---



###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Carregando as bibliotecas básicas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais

## Bibliotecas para treinamento/avaliação de modelos
from sklearn.model_selection import RepeatedKFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score, cross_val_predict, GridSearchCV
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

## Bibliotecas para converter variáveis categóricas (strings) para numéricas
from sklearn.preprocessing import OrdinalEncoder 

sns.set()


In [ ]:
## Carregando os dados
df = pd.read_csv("https://drive.google.com/uc?export=view&id=1hSiFQPybmELwtIzbByASRxKp0TczCpn6")
df  

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

In [ ]:
## Remoção dos dados duplicados
df.drop_duplicates(keep='last').shape
df = df.drop_duplicates(keep='last')

A coluna *'DiabetesRisk'* contém a classificação de cada instância. Vamos avaliar a distribuição de classes do problema.

In [ ]:
## Distribuição do atributo alvo
plt.hist(df['DiabetesRisk'])
plt.title("Distribuição do atributo alvo")
plt.show()

In [ ]:
df['DiabetesRisk'].value_counts()


---


### Criando conjuntos de treino e teste para avaliação de modelos


Antes de iniciar o treinamento do modelo, lembre-se que é recomendado sempre reservar uma porção dos dados para teste, a qual somente será utilizada para avaliação do modelo final (após todo o processo de treinamento e otimização de hiperparâmetros).

Vamos fazer esta divisão, separando 20% para teste. Entretanto, primeiro precisamos dividir os dados entre atributos (X) e classe (y). Também iremos codificar os valores categóricos em inteiros a fim de ampliar as opções de algoritmos que podemos utilizar no treinamento dos modelos.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['DiabetesRisk'], axis=1)
y = df['DiabetesRisk'].values

In [ ]:
## Codifica variáveis categóricas usando inteiros. 
## Automaticamente detecta as categorias a partir dos dados. 
## Neste caso, codifica Yes/No -> 1/0
encoder = OrdinalEncoder(dtype=np.int64)
encoder.fit(X)
X = encoder.transform(X)

Faremos o mapeamento das classes Positive/Negative para 1/0. Por padrão, as funções de avaliação assumem que a classe 1 é a positiva/de interesse.

In [ ]:
## substitui 'Negative' por 0, 'Positive' por 1
y = np.array([0 if y=='Negative' else 1 for y in y]) 

In [ ]:
## Faz a divisão entre treino (80%) e teste (20%).
## O conjunto de treino representa os dados que serão usados
## ao longo do desenvolvimento do modelo

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42) 

---


## Comparação de modelos com intervalos de confiança

Para gerar intervalos de confiança para uma determinada métrica e diferentes modelos de aprendizado supervisionado, vamos utilizar a estratégia de bootstrap. A construção dos intervalos de confiança é realizada conforme o que foi estudado em aula:


*   B amostras de bootstrap aleatórias são geradas, 
*   uma estimativa de desempenho é calculada a partir de cada amostra de bootstrap, 
*   todas as estimativas de desempenho de bootstrap B são ordenadas do valor mais baixo para o mais alto, 
*   o intervalo de confiança é construído selecionando os limites  [Percentil 100 x (α/2), Percentil 100 x (1 - α/2) ], para α = 0.05 (confiança de 95%) a partir da distribuição gerada

É importante notar que o procedimento acima é realizado para uma avaliação mais robusta de desempenho do modelo. No caso de ser feito otimização de hiperparâmetros, o processo pode estar dentro do loop dos boostraps, ou ainda, o processo de boostrap pode ser usado com diferentes configurações de hiperparâmetros.

In [ ]:
from sklearn.utils import resample

## configura o bootstrap. Serão realizadas 1000 iterações com reposição.
## as instâncias amostradas serão usadas para treino 
n_iterations = 1000
n_size = int(len(X_train))

In [ ]:
## executa o bootstrap
stats_dt = list()
stats_knn = list()

for i in range(n_iterations):
	# prepara treino e validação do bootstrap. as instâncias amostradas vão para partição de treino,
	# e as demais para a de validação (out-of-bag)deste bootstrap 
	train = resample(X_train, n_samples=n_size,random_state=i,replace=True)
	valid = np.array([x for x in X_train if x.tolist() not in train.tolist()])
 
	#treina o modelo de DT
	model_dt = DecisionTreeClassifier(max_depth=5, class_weight='balanced',random_state=42)
	model_dt.fit(train[:,:-1], train[:,-1])
 
	# avalia o modelo de DT
	predictions_dt = model_dt.predict(valid[:,:-1])
	score_dt = metrics.accuracy_score(valid[:,-1], predictions_dt)
	stats_dt.append(score_dt)
 
 	#treina o modelo de KNN
	model_knn = KNeighborsClassifier(n_neighbors=5)
	model_knn.fit(train[:,:-1], train[:,-1])

	# avalia o modelo de KNN
	predictions_knn = model_knn.predict(valid[:,:-1])
	score_knn = metrics.accuracy_score(valid[:,-1], predictions_knn)
	stats_knn.append(score_knn)

In [ ]:
# plota histograma dos scores
plt.figure(figsize=(10,7))
plt.subplot(211)
plt.hist(stats_dt)
plt.title("Distribuição de desempenho para árvore de decisão")


plt.subplot(212)
plt.hist(stats_knn)
plt.title("Distribuição de desempenho para KNN")

plt.show()

In [ ]:
## Intervalos de confiança. Nível de confiança dado por alpha
alpha = 0.95

## Árvore de Decisão
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats_dt, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats_dt, p))
print('Árvore de decisão. Média: %.1f' % np.mean(stats_dt))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))


## KNN
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats_knn, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats_knn, p))
print('KNN. Média: %.1f' % np.mean(stats_knn))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))

---


## Comparação de modelos com teste de hipótese

Vamos utilizar os métodos implementados na biblioteca do Python [mlxtend](http://rasbt.github.io/mlxtend/) (machine learning extesions) para fazer uma comparação estatística de modelos usando testes de hipótese. A bilbioteca disponibiliza os testes estudados em aula: teste de McNemar e teste t de sutdent com 5x2 CV para comparar dois classificadores, teste Q de Cochran para comparar múltiplos classificadores. Abaixo vamos exemplificar o uso de cada um deles. 

Iniciamos instalando a biblioteca.

In [ ]:
!pip install mlxtend 


#### Comparação entre dois classificadores

Esta seção exemplifica o uso do Teste t pareado modificado para combinar dois classificadores. Utilizaremos o método [paired_ttest_5x2cv](http://rasbt.github.io/mlxtend/user_guide/evaluate/paired_ttest_5x2cv/).

In [ ]:
from mlxtend.evaluate import paired_ttest_5x2cv

clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = DecisionTreeClassifier(random_state=42,max_depth=4)

t, p = paired_ttest_5x2cv(estimator1=clf1,
                          estimator2=clf2,
                          X=X, y=y,
                          random_seed=1,
                          scoring='f1')


In [ ]:
# sumarizando
print('P-valor: %.3f, t-Statistic: %.3f' % (p, t))
# interpretando o resultado
if p <= 0.05:
	print('Diferença no desempenho médio provavelmente é real (significativa)')
else:
	print('Os algoritmos provavelmente possuem o mesmo desempenho médio')

Este notebook se concentra em exemplificar o uso dos testes de hipótese no contexto de aprendizado de máquina. Antes de iniciar a comparação estatística, poderíamos ter realizado uma otimização de hiperparâmetros para entender o desempenho dos algoritmos em diferentes configurações, escolhendo a melhor. A análise estatística é comumente usada para avaliar o desempenho final dos modelos. Para fins de simplificação, esta etapa não foi executada neste notebook.

#### Comparação entre múltiplos classificadores

A comparação entre múltiplos modelos pode ser feita com o teste de Cochrans. A biblioteca mlxtend disponibiliza uma implementação do método, [veja neste link](http://rasbt.github.io/mlxtend/user_guide/evaluate/cochrans_q/)

In [ ]:
from mlxtend.evaluate import cochrans_q
from mlxtend.evaluate import mcnemar_table
from mlxtend.evaluate import mcnemar
from sklearn.svm import SVC

clf1 = KNeighborsClassifier(n_neighbors=9)
clf1.fit(X_train,y_train)
y_clf1 = clf1.predict(X_test)

clf2 = DecisionTreeClassifier(random_state=42,max_depth=4)
clf2.fit(X_train,y_train)
y_clf2 = clf2.predict(X_test)

clf3 = SVC(kernel='rbf',C=0.1,class_weight='balanced')
clf3.fit(X_train,y_train)
y_clf3 = clf3.predict(X_test)

q, p_value = cochrans_q(y_test, 
                        y_clf1, 
                        y_clf2, 
                        y_clf3)

# sumarizando
print('P-valor: %.3f, Q: %.3f' % (p_value, q))
# interpretando o resultado
if p_value <= 0.05:
	print('Diferença no desempenho médio provavelmente é real (significativa)')
else:
	print('Os algoritmos provavelmente possuem o mesmo desempenho médio')

Caso o teste de Cochrans entre os múltiplos modelos retorne um p-valor significativo, precisamos fazer testes pos-hoc par-a-par para investigar quais modelos diferem significativamente entre si em termos de desempenho preditivo. Uma possibilidade de teste a ser aplicado nesta estapa é o McNemar, indicado para comparação entre dois modelos.

In [ ]:
chi2, p_value = mcnemar(mcnemar_table(y_test, 
                                      y_clf1, 
                                      y_clf2),
                        corrected=False)

print('KNN vs DT. McNemar\'s Chi^2: %.3f' % chi2)
print('KNN vs DT. McNemar\'s p-value: %.3f' % p_value)


chi2, p_value = mcnemar(mcnemar_table(y_test, 
                                      y_clf1, 
                                      y_clf3),
                        corrected=False)

print('KNN vs SVC. McNemar\'s Chi^2: %.3f' % chi2)
print('KNN vs SVC. McNemar\'s p-value: %.3f' % p_value)

chi2, p_value = mcnemar(mcnemar_table(y_test, 
                                      y_clf2, 
                                      y_clf3),
                        corrected=False)

print('DT vs SVC. McNemar\'s Chi^2: %.3f' % chi2)
print('DT vs SVC. McNemar\'s p-value: %.3f' % p_value)

Enquanto este notebook se propôs a introduzir alguns testes estatísticos, a biblioteca mlxtend possui diversos outros métodos disponíveis para comparação de modelos em Aprendizado de Máquina. Veja a d[ocumentação](http://rasbt.github.io/mlxtend/#) da biblioteca e os exemplos de aplicação dos demais testes de hipótese.